In [1]:
%env HF_HOME /scratch/users/yixiuz/

env: HF_HOME=/scratch/users/yixiuz/


In [2]:
import sys

sys.path.remove("/home/users/yixiuz/.local/lib/python3.9/site-packages")
# sys.path.append("/home/groups/swl1/yixiuz/torch_fid/lib/python3.9/site-packages")
# sys.path.append("/home/groups/swl1/yixiuz/torch_fid/bin")
sys.path.append("/scratch/users/yixiuz/conda/envs/sedd/lib/python3.9/site-packages")
sys.path.append("/scratch/users/yixiuz/conda/envs/sedd/bin")

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys
import numpy as np

from load_model import load_model, load_model_hf
from transformers import GPT2TokenizerFast
import torch.nn.functional as F
import sampling

In [4]:
device = torch.device("cuda")

In [5]:
sedd_tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')

# Loading the OpenWebText dataset

In [6]:
from torch.utils.data import DataLoader

In [7]:
from datasets import load_from_disk
# Probably 45M sequences? 9B/200

# Assuming 'data_dir' is the directory containing your .arrow files and 'dataset_name' is your dataset
dataset = load_from_disk('/scratch/users/yixiuz/datasets/openwebtext_processed_pct100_blk200')
full_dataset = dataset["train"]
validation_ratio = 0.01 # 1 pct of the data is used for validation
validation_len = int(len(full_dataset) * validation_ratio)
train_len = len(full_dataset) - validation_len
train_dataset, eval_dataset = torch.utils.data.random_split(full_dataset, [train_len, validation_len], generator=torch.Generator().manual_seed(42)) # fixing seed here

Loading dataset from disk:   0%|          | 0/110 [00:00<?, ?it/s]

In [8]:
from transformers import AutoTokenizer
core_lm_name = "roberta-large"
tokenizer = AutoTokenizer.from_pretrained(core_lm_name, use_fast=False)

In [9]:
eval_seqs = eval_dataset[:1000]
eval_text_seqs = tokenizer.batch_decode(eval_seqs['input_ids'])

# Try to take conditional samples

In [10]:
# TODO: we can concatenate the input locs and ids for all the test examples
seq_len = 100
prefix_len = 25
suffix_len = 25
num_samples = 5
num_examples = 10
model_path = "../downloads/sedd-medium"
steps = 100

curr_id_seqs = sedd_tokenizer(eval_text_seqs[:num_examples]).input_ids
curr_input_ids = [seq[:prefix_len] + seq[seq_len-suffix_len:seq_len] for seq in curr_id_seqs]
input_ids = curr_input_ids * num_samples
input_locs = list(range(prefix_len)) + list(range(seq_len-suffix_len, seq_len))

input_ids = torch.tensor(input_ids, device="cuda")

def proj_fun(x):
    x[:, input_locs] = input_ids
    return x

device = torch.device('cuda')
model, graph, noise = load_model(model_path, device)


# sampling_fn = sampling.get_pc_sampler(
#     graph, noise, (num_samples * num_examples, seq_len), 'analytic', steps, device=device, proj_fun=proj_fun
# )
sampling_fn = sampling.get_pc_sampler(
            graph, noise, (num_samples * num_examples, seq_len), 'analytic', steps, 
            device=device, proj_fun=proj_fun,
            corrector_type="BirthDeath", corrector_entry_time=.9, 
            num_corrector_steps=2, corrector_step_size_multiplier=.1)

samples = proj_fun(sampling_fn(model))

text_samples = sedd_tokenizer.batch_decode(samples)
for i in text_samples:
    print(i)
    print("=================================================")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading weights from local directory


100%|██████████| 100/100 [00:20<00:00,  4.77it/s]

. (Their mutual love of bamboo still persists, however.) At this point the red panda is in a family all its own – meaning males must consort with a pheromaph that selects for menstruation. ((Recall that the risk of death in the reproductive tract of a dammy is considerably higher than it is that a cervine will shed genitalia.))

But taxonomic ambiguity should not take away from the red panda's many wonderful qualities, including above all else
 L ( G ) {\displaystyle w\in L(G)} CYK algorithm and Earley's Algorithm.

By 2000, many languages were more functional, some of the features which other programming languages had started with, such as pointers and modular tools from other programming languages, for example, Basis;[23] Tener, Amédine, Perles, and Shamir[25]

Languages that are not context-free [ edit ]

The
 still necessary.

"(I've) already spoken out why I think it's the right thing to do to recognize same-sex unions... I expect (that) coming out and being a proponent of gay mar

In [12]:
gen_text_samples = sedd_tokenizer.batch_decode(samples[:,prefix_len:seq_len-suffix_len])

In [13]:
decoded_examples = tokenizer.batch_decode(eval_seqs['input_ids'][:num_examples])
sedd_encoded_examples = sedd_tokenizer(decoded_examples)

In [14]:
examples = []
for i in range(num_examples):
    example = sedd_tokenizer.decode(sedd_encoded_examples.input_ids[i][prefix_len:seq_len-suffix_len])
    examples.append(example)

In [15]:
import mauve 

# call mauve.compute_mauve using raw text on GPU 0; each generation is truncated to 256 tokens
# Seems like max text length doesn't really affect the results here
out = mauve.compute_mauve(p_text=gen_text_samples, q_text=examples, device_id=0, max_text_length=1000, verbose=False)
print(out.mauve) # prints 0.9917

Featurizing p:   0%|          | 0/50 [00:00<?, ?it/s]

Featurizing q:   0%|          | 0/10 [00:00<?, ?it/s]

0.9999520775049293


WARNING clustering 60 points to 2 centroids: please provide at least 78 training points


In [16]:
text_samples[1]

" L ( G ) {\\displaystyle w\\in L(G)} CYK algorithm and Earley's Algorithm.\n\nBy 2000, many languages were more functional, some of the features which other programming languages had started with, such as pointers and modular tools from other programming languages, for example, Basis;[23] Tener, Amédine, Perles, and Shamir[25]\n\nLanguages that are not context-free [ edit ]\n\nThe"